In [54]:
from github import Github
import pandas as pd
import os
from github import Github, Auth
import re
from dotenv import load_dotenv
from tqdm import tqdm
import time

In [2]:
GITHUB_TOKEN = "YOUR_TOKEN"
g = Github(auth=Auth.Token(GITHUB_TOKEN))
user = g.get_user()
print(user.login)

Shhhyuki


In [49]:
df_repos = pd.read_csv('filtered_repos.csv')

In [58]:
def save_batch(data, output_file):
    if not data:
        return
    df = pd.DataFrame(data)
    df.to_csv(
        output_file,
        mode="a",
        header=not os.path.exists(output_file), 
        index=False
    )
    print(f"Saved {len(data)} commits to {output_file}")

def wait_for_quota():
    rate = g.get_rate_limit().rate
    print(f"remaining request: {rate.remaining}/{rate.limit}")
    if rate.remaining < 100:  
        reset_time = rate.reset.timestamp()
        sleep_for = int(reset_time - time.time()) + 10
        if sleep_for > 0:
            print(f"Quota low, sleeping for {sleep_for//60} minutes...")
            time.sleep(sleep_for)

In [12]:
# loading repos
repos = []
print(df_repos.shape)
for name in df_repos["name"]:
    try:
        repo = g.get_repo(name)
        repos.append(repo)
        print(f"Loaded repo: {repo.full_name}")
    except Exception as e:
        print(f"Failed to load repo {name}: {e}")

(297, 8)
Loaded repo: puppetlabs/puppetlabs-apache
Loaded repo: wikimedia/operations-puppet
Loaded repo: dev-sec/puppet-os-hardening
Loaded repo: theforeman/puppet-puppet
Loaded repo: puppetlabs/puppetlabs-puppetdb
Loaded repo: voxpupuli/puppet-zabbix
Loaded repo: voxpupuli/puppet-logstash
Loaded repo: voxpupuli/puppet-prometheus
Loaded repo: wazuh/wazuh-puppet
Loaded repo: voxpupuli/puppet-php
Loaded repo: voxpupuli/puppet-openvpn
Loaded repo: voxpupuli/puppet-systemd
Loaded repo: voxpupuli/puppet-gitlab
Loaded repo: pcfens/puppet-filebeat
Loaded repo: puppetlabs-toy-chest/puppetlabs-openstack
Loaded repo: jippi/puppet-php
Loaded repo: voxpupuli/puppet-icinga2
Loaded repo: openstack/puppet-cinder
Loaded repo: Icinga/puppet-icinga2-legacy
Loaded repo: alphagov/govuk-puppet
Loaded repo: example42/puppi
Loaded repo: openstack/puppet-heat
Loaded repo: openstack/puppet-openstack-integration
Loaded repo: example42/puppet-tp
Loaded repo: goodrobots/maverick
Loaded repo: ocf/puppet
Loaded rep

AttributeError: 'list' object has no attribute 'shape'

In [67]:
output_file  = "xcm.csv"
commits_withxcm_data = []
commit_batch_size = 100
ISSUE_REGEX = r"#\d+" #to catch github issues starts with #
issue_cache = {}

# load last saved commit message
if os.path.exists(output_file):
    last_row = pd.read_csv(output_file).iloc[-1]
    last_repo = last_row["repo"]
    last_sha = last_row["sha"]
    print(f" Resume from repo={last_repo}, commit={last_sha}")
else:
    last_repo, last_sha = None, None

# turn on resume mode if we had a last repo
resume_mode = last_repo is not None
resumed = False

for repo_idx, repo in enumerate(repos):
    print(f"\n Processing repo {repo_idx+1}: {repo.full_name}")

    # check if it's the checkpoint
    if resume_mode and not resumed:
        if repo.full_name != last_repo:
            continue
            
    # count total commit in the repo
    total_commits_num = int(df_repos[df_repos['name'] == repo.full_name]['commits'].iloc[0])
    wait_for_quota()

    for commit in tqdm(repo.get_commits(), total=total_commits_num, desc=f"{repo.full_name} commits"):
        try:
            if resume_mode and not resumed:
                if commit.sha == last_sha:
                    resumed = True 
                    print("checkpoint found, start mining...")
                continue

            files = commit.files
            for f_idx, f in enumerate(files):
                if f.filename.endswith(".pp"):
                    # get commit message
                    msg = commit.commit.message

                    # find issue in commit message, save issue num to cache to save requests to git
                    issue_summary = ""
                    issue_ids = re.findall(ISSUE_REGEX, msg)
                    for issue_id in issue_ids:
                        num = int(issue_id[1:])
                        if num not in issue_cache:
                            try:
                                issue_cache[num] = repo.get_issue(num).title
                            except Exception:
                                issue_cache[num] = ""
                        issue_summary += " " + issue_cache[num]
                        
                    # merge commit message and issue summmary into extend commit message
                    xcm = msg + issue_summary
                    commits_withxcm_data.append({
                        "repo": repo.full_name,
                        "sha": commit.sha,
                        "message": msg,
                        "file": f.filename,
                        "author": commit.commit.author.name,
                        "date": commit.commit.author.date,
                        "xcm": xcm
                    })

            # save every 100 commits
            if len(commits_withxcm_data) >= commit_batch_size:
                save_batch(commits_withxcm_data, output_file)
                commits_withxcm_data = []
                wait_for_quota()

        except Exception as e:
            print(f" Error in repo {repo.full_name}: {e}")
            print(f" saving checkpoint...")
            save_checkpoint(repo_idx, )

save_batch(commits_withxcm_data, output_file)

 Resume from repo=Stanisz1/03-puppet-master-task-03, commit=674a65d8dca3a8a7feedfd2688b6a2fca43bc0c6

 Processing repo 1: puppetlabs/puppetlabs-apache

 Processing repo 2: wikimedia/operations-puppet

 Processing repo 3: dev-sec/puppet-os-hardening

 Processing repo 4: theforeman/puppet-puppet

 Processing repo 5: puppetlabs/puppetlabs-puppetdb

 Processing repo 6: voxpupuli/puppet-zabbix

 Processing repo 7: voxpupuli/puppet-logstash

 Processing repo 8: voxpupuli/puppet-prometheus

 Processing repo 9: wazuh/wazuh-puppet

 Processing repo 10: voxpupuli/puppet-php

 Processing repo 11: voxpupuli/puppet-openvpn

 Processing repo 12: voxpupuli/puppet-systemd

 Processing repo 13: voxpupuli/puppet-gitlab

 Processing repo 14: pcfens/puppet-filebeat

 Processing repo 15: puppetlabs-toy-chest/puppetlabs-openstack

 Processing repo 16: jippi/puppet-php

 Processing repo 17: voxpupuli/puppet-icinga2

 Processing repo 18: openstack/puppet-cinder

 Processing repo 19: Icinga/puppet-icinga2-lega

Stanisz1/03-puppet-master-task-03 commits:  85%|████████████▋  | 61/72 [00:01<00:00, 40.92it/s]

checkpoint found, start mining...


Stanisz1/03-puppet-master-task-03 commits: 100%|███████████████| 72/72 [00:02<00:00, 31.97it/s]
